In [1]:
using CSV
using Dates
using TimeZones
using DataFramesMeta
using Statistics
using StatsBase

include("../data/datacatalog.jl");

In [3]:
ercotDAPrices = reduce(
    vcat,
    [
        CSV.read("../Data/ERCOT DA Energy Price 2015.csv", DataFrame; dateformat=ercotDAEnergyPrice.dateformat, types=ercotDAEnergyPrice.schema, validate=true),
        CSV.read("../Data/ERCOT DA Energy Price 2016.csv", DataFrame; dateformat=ercotDAEnergyPrice.dateformat, types=ercotDAEnergyPrice.schema, validate=true),
        CSV.read("../Data/ERCOT DA Energy Price 2017.csv", DataFrame; dateformat=ercotDAEnergyPrice.dateformat, types=ercotDAEnergyPrice.schema, validate=true),
        CSV.read("../Data/ERCOT DA Energy Price 2018.csv", DataFrame; dateformat=ercotDAEnergyPrice.dateformat, types=ercotDAEnergyPrice.schema, validate=true),
        CSV.read("../Data/ERCOT DA Energy Price 2019.csv", DataFrame; dateformat=ercotDAEnergyPrice.dateformat, types=ercotDAEnergyPrice.schema, validate=true),
        CSV.read("../Data/ERCOT DA Energy Price 2020.csv", DataFrame; dateformat=ercotDAEnergyPrice.dateformat, types=ercotDAEnergyPrice.schema, validate=true),
        CSV.read("../Data/ERCOT DA Energy Price 2021.csv", DataFrame; dateformat=ercotDAEnergyPrice.dateformat, types=ercotDAEnergyPrice.schema, validate=true),
        CSV.read("../Data/ERCOT DA Energy Price 2022.csv", DataFrame; dateformat=ercotDAEnergyPrice.dateformat, types=ercotDAEnergyPrice.schema, validate=true),
        CSV.read("../Data/ERCOT DA Energy Price 2023.csv", DataFrame; dateformat=ercotDAEnergyPrice.dateformat, types=ercotDAEnergyPrice.schema, validate=true),
        CSV.read("../Data/ERCOT DA Energy Price 2024.csv", DataFrame; dateformat=ercotDAEnergyPrice.dateformat, types=ercotDAEnergyPrice.schema, validate=true),
    ]
)
ercotDAPrices = @select ercotDAPrices :interval_start_cst :spp
misoDAPrices = reduce(
    vcat,
    [
        CSV.read("../Data/MISO DA Energy Price by Hub 2015.csv", DataFrame; dateformat=misoDAPriceByHub.dateformat, types=misoDAPriceByHub.schema, validate=true),
        CSV.read("../Data/MISO DA Energy Price by Hub 2016.csv", DataFrame; dateformat=misoDAPriceByHub.dateformat, types=misoDAPriceByHub.schema, validate=true),
        CSV.read("../Data/MISO DA Energy Price by Hub 2017.csv", DataFrame; dateformat=misoDAPriceByHub.dateformat, types=misoDAPriceByHub.schema, validate=true),
        CSV.read("../Data/MISO DA Energy Price by Hub 2018.csv", DataFrame; dateformat=misoDAPriceByHub.dateformat, types=misoDAPriceByHub.schema, validate=true),
        CSV.read("../Data/MISO DA Energy Price by Hub 2019.csv", DataFrame; dateformat=misoDAPriceByHub.dateformat, types=misoDAPriceByHub.schema, validate=true),
        CSV.read("../Data/MISO DA Energy Price by Hub 2020.csv", DataFrame; dateformat=misoDAPriceByHub.dateformat, types=misoDAPriceByHub.schema, validate=true),
        CSV.read("../Data/MISO DA Energy Price by Hub 2021.csv", DataFrame; dateformat=misoDAPriceByHub.dateformat, types=misoDAPriceByHub.schema, validate=true),
        CSV.read("../Data/MISO DA Energy Price by Hub 2022.csv", DataFrame; dateformat=misoDAPriceByHub.dateformat, types=misoDAPriceByHub.schema, validate=true),
        CSV.read("../Data/MISO DA Energy Price by Hub 2023.csv", DataFrame; dateformat=misoDAPriceByHub.dateformat, types=misoDAPriceByHub.schema, validate=true),
    ]
)
misoDAPrices = @rsubset misoDAPrices :node == "MINN.HUB";

In [4]:
@show summarystats(ercotDAPrices.spp)
std(ercotDAPrices.spp)

summarystats(ercotDAPrices.spp) = Summary Stats:
Length:         86766
Missing Count:  0
Mean:           46.146536
Minimum:        -2.850000
1st Quartile:   17.560000
Median:         22.540000
3rd Quartile:   32.380000
Maximum:        8995.260000



301.84383155337144

In [5]:
@show summarystats(misoDAPrices.lmp)
std(misoDAPrices.lmp)

summarystats(misoDAPrices.lmp) = Summary Stats:
Length:         78864
Missing Count:  0
Mean:           27.127952
Minimum:        -6.490000
1st Quartile:   18.240000
Median:         22.910000
3rd Quartile:   31.050000
Maximum:        606.820000



18.169805691848886

In [6]:
function topBottomSpread(num::Int, prices::AbstractArray)
    prices = sort(prices) # This should make a copy
    maxTotal = sum(prices[24-num:end])
    minTotal = sum(prices[1:num])

    spread = (maxTotal - minTotal) / num
    return spread
end

topBottomSpread (generic function with 1 method)

In [7]:
@chain ercotDAPrices begin
    @rtransform :year = year(:interval_start_cst)
    @rtransform :dayofyear = dayofyear(:interval_start_cst)
    # @rsubset begin
        # :year == 2016
        # :dayofyear <= 13
    # end
    @groupby [:year, :dayofyear]
    filter(:spp => spp -> length(spp) == 24, _)
    @combine :spread = topBottomSpread(4, :spp)
    @groupby :year
    @combine begin
        :mean_spread = mean(:spread)
        :max_spread = maximum(:spread)
        :min_spread = minimum(:spread)
        :std_spread = std(:spread)
    end
end

Row,year,mean_spread,max_spread,min_spread,std_spread
,Int64,Float64,Float64,Float64,Float64
1,2015,39.8897,1335.29,10.35,82.6142
2,2016,33.1284,170.868,9.99,22.9822
3,2017,31.9966,176.265,10.385,14.7826
4,2018,67.6872,1482.17,13.4725,164.541
5,2019,115.82,3695.05,9.2525,382.346
6,2020,40.114,1247.53,7.9575,78.1145
7,2021,153.133,9664.95,12.2325,762.912
8,2022,120.266,2131.42,13.8625,193.09
9,2023,190.187,4872.99,14.105,534.062


In [8]:
@chain misoDAPrices begin
    @rtransform :year = year(:interval_start_est)
    @rtransform :dayofyear = dayofyear(:interval_start_est)
    # @rsubset begin
        # :year == 2016
        # :dayofyear <= 13
    # end
    @groupby [:year, :dayofyear]
    filter(:lmp => lmp -> length(lmp) == 24, _)
    @combine :spread = topBottomSpread(4, :lmp)
    @groupby :year
    @combine begin
        :mean_spread = mean(:spread)
        :max_spread = maximum(:spread)
        :min_spread = minimum(:spread)
        :std_spread = std(:spread)
    end
end

Row,year,mean_spread,max_spread,min_spread,std_spread
,Int64,Float64,Float64,Float64,Float64
1,2015,24.3462,98.6175,10.3425,9.56883
2,2016,23.0418,84.3775,7.7875,10.197
3,2017,24.6419,60.7775,9.61,8.46337
4,2018,26.4375,144.108,9.145,13.5412
5,2019,19.7788,103.905,8.2925,8.85033
6,2020,18.4222,53.395,6.095,8.23945
7,2021,40.5169,533.983,8.8,43.5971
8,2022,56.5464,145.785,13.7425,25.411
9,2023,36.3451,187.325,10.78,21.4474


[14.77, 14.92, 15.19, 16.26, 16.28, 16.56, 16.72, 16.76, 17.37, 17.74, 18.16, 18.68, 18.85, 19.3, 20.05, 20.54, 20.85, 21.03, 21.04, 21.21, 21.58, 21.87, 25.98, 32.51]


15.502500000000001